# Created by Stezar Priansya  09/05/2018

In [2]:
# Created By Stezar Priansya
# ini dari azure
import pandas as pd
import numpy as np

In [3]:
owner = pd.read_csv("Pet/Owners.csv")
pet = pd.read_csv("Pet/Pets.csv")
detail = pd.read_csv("Pet/ProceduresDetails.csv")
history = pd.read_csv("Pet/ProceduresHistory-1.csv")

## Nomor 1 : Join 4 Tabel

In [4]:
# Nomor 1 : Join 4 tabel
df = pd.merge(history, pet, how='inner', on='PetID')
df = pd.merge(df, owner, how='inner',on='OwnerID')
df = pd.merge(df, detail, how='inner', on=['ProcedureType', 'ProcedureSubCode'])

In [5]:
# untuk mencari dimensi
df.shape

(41, 18)

## Nomor 2 : Procedure type terbanyak

In [6]:
# Nomor 2 : prosedur terbanyak
prosedur_banyak = df.groupby('ProcedureType').count()
prosedur_banyak[prosedur_banyak.PetID == max(prosedur_banyak.PetID)]['PetID']

ProcedureType
VACCINATIONS    29
Name: PetID, dtype: int64

Dari cara diatas didapat bahwa prosedur terbanyak adalah vaksinasi sebanyak 29 kali

## Nomor 3 : pasien yang datang ke dokter lebih dari 3 kali

In [7]:
# Nomor 3 : pasien yang datang ke dokter lebih dari 3 kali
dokter = df.groupby('OwnerID').count()
person = owner[owner.OwnerID == dokter[dokter.PetID > 3].index[0]]
print('Yang datang ke dokter lebih dari 3 kali adalah '+owner.iloc[person.index[0]].Name+' '+owner.iloc[person.index[0]].Surname)

Yang datang ke dokter lebih dari 3 kali adalah Lee McKenzie


In [8]:
# convert to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['day_of_week'] = df['Date'].dt.weekday_name

In [9]:
df['day_type'] = np.where((df['day_of_week'] == "Sunday") | (df['day_of_week'] == "Saturday"), "Weekend", "Weekday")

## Nomor 4 : Nama yang paling banyak berkunjung ke dokter

In [10]:
# Nomor 4 : yang paling banyak berkunjung ke dokter
orang_terbanyak = owner[owner.OwnerID == df[df.day_type == 'Weekday'].groupby('OwnerID').count().sort_values(by=['day_type'], ascending=False).head(1).index[0]]
print('Yang datang paling banyak ke dokter '+owner.iloc[orang_terbanyak.index[0]].Name+' '+owner.iloc[orang_terbanyak.index[0]].Surname)
# print(orang_terbanyak['Name']+' '+orang_terbanyak['Surname'])

Yang datang paling banyak ke dokter Lee McKenzie


## Nomor 5 : Total spending Jerry dan Jessica

In [11]:
# Nomor 5: total spending jerry dan jessica
df[(df.Name_y == 'Jerry') | (df.Name_y == 'Jessica')].groupby(['OwnerID', 'Name_y']).sum()['Price']

OwnerID  Name_y 
1070     Jessica     25
5806     Jerry      400
Name: Price, dtype: int64

Dari cara diatas didapat bahwa biaya dari Jessica adalah 25 dan Jerri adalah 400

## Nomor 6 : Nama Hewan yang paling sering di vaksinasi

In [12]:
# Nomor 6: Nama Hewan yang paling sering di vaksinasi
# df.groupby('PetID').count().sort_values(by='Name_x', ascending=False).head(1).index[0]
# pet[pet.PetID == df.groupby('PetID').count().sort_values(by='Name_x', ascending=False).head(1).index[0]]
row_pet = pet[pet.PetID == df[df.ProcedureType == 'VACCINATIONS'].groupby('PetID').count().sort_values(by=['day_type'], ascending=False).head(1).index[0]]
row_pet['Name']

57    Bonaparte
Name: Name, dtype: object

Dari data diatas dapat dilihat bahwa yang paling banyak di vaksinasi adalah Bonaparte

In [13]:
# mencari rata2 umur
rata_umur = df.groupby(['Kind']).mean()['Age']
rata_umur

Kind
Cat       6.833333
Dog       6.956522
Parrot    6.666667
Name: Age, dtype: float64

## Nomor 7 : 5 anjing terbanyak dengan simpangan umur terjauh

In [14]:
# Nomor 7 : 5 anjing terbanyak dengan simpangan umur terjauh
anjing = df[df.Kind == 'Dog'].copy()
anjing = anjing.groupby(['PetID', 'Name_x']).mean()
anjing['selisih_umur'] = abs(anjing['Age']-rata_umur['Dog'])
# df.loc[df['Kind'] == 'Dog', 'selisih_umur'] = abs(anjing['Age']-rata_umur['Dog'])
anjing.sort_values(by=['selisih_umur'], ascending=False).head(5)[['Age', 'selisih_umur']]
# pet[pet['PetID'].isin(anjing.index)]

,,Age,selisih_umur
PetID,Name_x,,
P9-3625,Cuddles,14.0,7.043478
Q9-1565,Scout,14.0,7.043478
P0-1725,Lily,0.0,6.956522
L4-6292,Bright,13.0,6.043478
P2-7342,Cuddles,13.0,6.043478


Yang paling banyak adalah Cuddles, Scout, Lily, Bright, Cuddles

## Nomor 8: Hewan terbanyak datang ke dokter waktu weekend

In [15]:
# Nomor 8: Hewan terbanyak datang ke dokter waktu weekend
df[df.day_type == 'Weekend'].groupby('Kind').count().sort_values(by=['day_type'], ascending=False).head(1)
# ini nama hewannya
# pet[pet.PetID == df[df.day_type == 'Weekend'].groupby('PetID').count().sort_values(by=['day_type'], ascending=False).head(1).index[0]]

,PetID,Date,ProcedureType,ProcedureSubCode,Name_x,Gender,Age,OwnerID,Name_y,Surname,StreetAddress,City,State,StateFull,ZipCode,Description,Price,day_of_week,day_type
Kind,,,,,,,,,,,,,,,,,,,
Dog,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12


Yang paling banyak berkunjung datang ke dokter waktu weekend adalah hewan Anjing

## Hewan 9 : Nama Hewan yang paling sering di vaksinasi

In [16]:
# Hewan 9 : Nama Hewan yang paling sering di vaksinasi
pet[pet.PetID == df[df.ProcedureType == 'VACCINATIONS'].groupby('PetID').count().sort_values(by=['day_type'], ascending=False).head(1).index[0]]

,PetID,Name,Kind,Gender,Age,OwnerID
57,J8-7429,Bonaparte,Dog,male,4,8215


Dari data diatas dapat dilihat bahwa yang paling banyak di vaksinasi adalah Bonaparte

## Hewan 10 : rata2 harga paling mahal dari procedure

In [17]:
# Hewan 10 : rata2 harga paling mahal dari procedure
mean_proc = df.groupby(['ProcedureType']).mean()
mean_proc.sort_values(by='Price', ascending=False).head(1)['Price']

ProcedureType
ORTHOPEDIC    267.5
Name: Price, dtype: float64

Jadi rata-rata paling mahal dari procedure adalah Orthopedic dengan rata-rata harga 267.5